# プログラム 8-1

## 人間のように「ふるまう」プログラム

In [ ]:
# リストのリストで迷路を表現する
maze = [
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
# 迷路のある場所がかべかどうか調べる
x = 0  # よこの位置(左端が0)
y = 1  # たての位置(上端が0)
maze[y][x]  # 0が道，1がかべ

In [ ]:
# 迷路を表示する関数を読み込む
!pip install mpb_lib -qU
from mpb_lib.maze import show_maze, show_route

In [ ]:
# 迷路を表示する
show_maze(maze)

### 迷路の探索プログラム

In [ ]:
# 上下左右を調べる関数
def find(x, y, maze, depth):
    # mazeのx,yの位置で上下左右が壁か道かを調べる関数

    result_list = []  # 戻り値のリストを空のリストで初期化
    # 上下左右を調べるため，リストのリストを使ってループ
    for txy in [[0, -1], [0, 1], [-1, 0], [1, 0]]:
        # 調べる位置
        tx = x + txy[0]
        ty = y + txy[1]
        if tx >= 0 and tx <= 9 and ty >= 0 and ty <= 9:
            # 調べたい位置が迷路の範囲内だった
            p = maze[ty][tx]
            if p == 0 or p > depth:
                # 通っていない道だった場合，リストに追加
                result_list += [[tx, ty]]
        
    return result_list


In [ ]:
# 関数を使って道を調べてみる
find(1, 3, maze, 1)

In [ ]:
# 迷路の探索に使う変数を初期化
stack = []  # 探索の対象となる座標を保存するリストを初期化
route = []  # 答え(最短経路)を保存するリストを初期化

x = 0  # 現在位置(x)
y = 1  # 現在位置(y)
depth = 2  # 探索の深さ(スタートからの距離)

# 迷路を初期化
maze = [
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 0, 1, 0, 0, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
 [1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]

In [ ]:
# 探索用アルゴリスム(1ターン)
# 結果を確認しながら，何度か実行してみてください

maze[y][x] = depth  # 迷路のリストに，探索深さを記録
route.append([x, y])  # 最短経路のリストに位置を記録
the_way = find(x, y, maze, depth)  # 今の位置から移動可能な場所を探す

if len(the_way) >= 1:
    # 移動可能な場所があった
    # 2番目以降をスタックに保存しておく
    for x_y in the_way[1:]:
        # 次の場所のx, 次の場所のy，routeの長さの順に記録
        stack.append([x_y[0], x_y[1], len(route)])
    # 0番目の場所に移動する
    x = the_way[0][0]
    y = the_way[0][1]
    depth += 1  # 探索深さを+1
else:
    # 移動可能な場所がなかった
    # スタックに記録があれば，そこに戻って探索を続ける
    if len(stack) > 0:
        # stackの最後の位置から，座標と探索深さを復元
        item = stack.pop() # stackの一番最後の要素を取り出してから削除
        x = item[0]
        y = item[1]
        route = route[:item[2]]  # ルートを巻き戻す
        depth = item[2] + 2  # 探索深さを巻き戻す
    else:
        print("【ゴールが見つかりました】")
    
show_maze(maze) # 現在の経過を表示

In [ ]:
# 結果を表示する
route

In [ ]:
# 迷路上に最短経路を表示する
show_route(maze, route)